# Multi node Distributed training with PyTorch

## Multi Node Application with PyTorch Distributed: Message Exchanging Example

In [ ]:
! chmod 700 cnn_part5_message_passing.sh 
! mpirun -np 2 -ppn 1 cnn_part5_message_passing.sh 

## Multi Node Application with PyTorch Distributed: Damage Level Example

Download the dataset to $SCRATCH

In [ ]:
! chmod 700 cnn_part5_copy_data.sh 
! cnn_part5_copy_data.sh /work2/10000/zw427/data.tar.gz $SCRATCH

In [ ]:
! chmod 700 cnn_part5_run_distributed.sh
! mpirun -np 2 -ppn 1 cnn_part5_run_distributed.sh 